In [66]:
import pandas as pd

In [67]:
argotario = pd.read_csv('../../../data/argumentation/argotario.tsv', sep='\t')[['Text', 'Topic', 'Intended Fallacy']]

In [68]:
argotario = argotario.rename(columns={'Text': 'argument', 'Intended Fallacy': 'fallacy type'})
argotario = argotario[~argotario['fallacy type'].isna()]

argotario['fallacy type'] = argotario['fallacy type'].replace({
'Red Herring': 'fallacy of relevance', 
'Appeal to Emotion': 'appeal to emotion',
'No Fallacy': 'no fallacy',
'Hasty Generalization': 'faulty generalization',
'Irrelevant Authority':'fallacy of credibility',
'Ad Hominem': 'ad hominem'
})



In [69]:
from Evaluate import Eval

In [70]:
evaluator_model_name = 'google/t5-v1_1-small'
ckpt_path = '../../../runs/train/model/ckp_25000.pth'
evaluator = Eval(
    ckpt_path=ckpt_path, 
    score_model_name=evaluator_model_name, 
    classify_model_name='../../../models/electra-base-mnli-M2/')  

In [ ]:
scores = []
fallacies = []
for i, item in argotario.iterrows():
    sentence = item['argument']
    score = evaluator.score(sentence)
    pred = evaluator.classify_as_fallacy(sentence)
    scores.append(score.item())
    fallacies.append(item['fallacy type'])
    #print(f"TOPIC: {item['Topic']}\nArgument: {sentence}\nScore: {score}\nPredicted Fallacy: {pred}\nActual Fallacy: {item['fallacy type']}\n===================================")
  
import numpy as np
fallacies = np.array(fallacies)
scores = np.array(scores)

In [ ]:
(fallacies=='no fallacy')

array([False, False, False, ...,  True,  True,  True])

In [ ]:
idx = np.where(fallacies!='no fallacy')[0]
import pandas as pd
pd.DataFrame(scores[idx]).describe()

,0
count,909.000000
mean,0.316970
std,0.421411
min,0.001639
25%,0.002873
50%,0.020316
75%,0.846987
max,0.998820


In [ ]:
#for score, fallacy_type in zip(scores, fallacies):
 #   if score > 0.8:
  #      print(score, fallacy_type)